# Load Packages

In [1]:
import pandas as pd
from utils.model import TutorAssignmentModel
import matplotlib.pyplot as plt
import seaborn as sns

# Set plot style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (10,6)

ModuleNotFoundError: No module named 'docplex'

# Load Data

In [ ]:
DATA_PATH = "./data/small_data.xlsx"

# Load specific sheets
new_student_info = pd.read_excel(DATA_PATH, sheet_name="New Students")
tutors = pd.read_excel(DATA_PATH, sheet_name="Tutor Information")
existing_student_info = pd.read_excel(DATA_PATH, sheet_name="Existing Students")
existing_student_info_filtered = existing_student_info[existing_student_info['active'] == True]

In [ ]:
new_student_info

# Scenario 1: Minimize total tutor used & maximize tutor preference

In [ ]:
# optimizer = TutorAssignmentModel(
#         new_students=new_student_info,tutor_info=tutors,
#         existing_students=existing_student_info_filtered,
#         beta= 0.9,
#         scenario=1
#         )
# optimizer.main_process()

## Generate reports for analysis

In [ ]:
path = './results/'
task_1_assignments = pd.read_csv(f'{path}scenario_1_assignment_results.csv')
task_1_preference_report = pd.read_csv(f'{path}scenario_1_preference_report.csv')
task_1_tutor_summary = pd.read_csv(f'{path}scenario_1_tutor_summary.csv')

## Visualisation

### Tutor Utilization



#### Bar chart: How many students are assigned to each tutor compared against tutor's max capacity

In [ ]:
plt.figure()
plt.bar(task_1_tutor_summary["TutorID"],task_1_tutor_summary["TotalAssigned"], label="Total Assigned", color="skyblue")
plt.plot(task_1_tutor_summary["TutorID"],task_1_tutor_summary["MaxCapacity"], 'r--', label="Max Capacity")
plt.xlabel("Tutor ID")
plt.ylabel("Students")
plt.title("Tutor Utilization vs. Max Capacity")
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

#### Bar Chart: Each tutor's assigned student stacked against remaining capacity

In [ ]:
task_1_tutor_summary

In [ ]:
# Calculate utility as fraction of max capacity
task_1_tutor_summary["UtilityPct"] = (task_1_tutor_summary["TotalAssigned"] / 
                                      task_1_tutor_summary["MaxCapacity"] * 100)

plt.figure(figsize=(10,6))

# Stacked bars
plt.bar(task_1_tutor_summary["TutorID"], task_1_tutor_summary["TotalAssigned"], 
        label="Assigned", color="skyblue")
plt.bar(task_1_tutor_summary["TutorID"], 
        task_1_tutor_summary["MaxCapacity"] - task_1_tutor_summary["TotalAssigned"], 
        bottom=task_1_tutor_summary["TotalAssigned"], 
        label="Remaining Capacity", color="lightgray")

# Overlay utility percentage on top of bars
for idx, row in task_1_tutor_summary.iterrows():
        plt.text(
                row["TutorID"], row["TotalAssigned"] + 0.2, 
                f'{row["UtilityPct"]:.0f}%', ha='center', va='bottom', fontsize=9, fontweight='bold')

plt.xlabel("Tutor ID")
plt.ylabel("Number of Students")
plt.title("Tutor Capacity vs Students Assigned (with Utility %)")
plt.legend()
plt.tight_layout()
plt.show()

#### Stacked Bar Chart: Total Student assigned to different Tutors browken down into Existing VS New Students

In [ ]:
plt.bar(task_1_tutor_summary["TutorID"], task_1_tutor_summary["Existing"], label="Existing", color="lightgray")
plt.bar(task_1_tutor_summary["TutorID"], task_1_tutor_summary["NewAssigned"], bottom=task_1_tutor_summary["Existing"], label="New Assigned", color="skyblue")
plt.xlabel("Tutor ID")
plt.ylabel("Number of Students")
plt.title("Task 1: Tutor Utilization (Existing + New)")
plt.legend()
plt.tight_layout()
plt.show()

### Free Capacity Distribution

#### Histogram showing discrete counts of tutors for each free capacity value

In [ ]:
# Count number of tutors per exact free capacity
summary_counts = task_1_tutor_summary["FreeCapacity"].value_counts().sort_index()

# Vertical bar plot
plt.figure(figsize=(8,5))
plt.bar(summary_counts.index, summary_counts.values, color="lightgreen", edgecolor="black")
plt.xlabel("Free Capacity")
plt.ylabel("Number of Tutors")
plt.title("Tutor Free Capacity Distribution (Discrete)")

plt.xticks(summary_counts.index)  # ensure all integer free capacities shown
plt.tight_layout()
plt.show()


### Preference Satistifaction by Centre

In [ ]:
task_1_preference_report

#### Bar Chart: Proportion of student–tutor assignments where the tutor’s preferred centre includes the student’s tuition centre

In [ ]:
pref_rate = task_1_preference_report["MatchPref"].mean() * 100
print(f"Overall Tutor Preference Satisfaction: {pref_rate:.1f}%")

pref_by_centre = task_1_preference_report.groupby("StudentCentre")["MatchPref"].mean().mul(100)
pref_by_centre.plot(kind='bar', color='orange')
plt.ylabel("% of Matches Meeting Tutor Preference")
plt.title("Preference Satisfaction by Tuition Centre")
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()


### Assignment Matrix

#### Visualises assignment of new students to different tutors (sanity check that 1 student - 1 tutor rule met)

In [ ]:
pivot = task_1_assignments.pivot_table(index="TutorID", columns="StudentID", aggfunc=len, fill_value=0)

plt.figure(figsize=(12,8))
sns.heatmap(pivot, cmap="Blues", cbar=True, linewidths=0.5, linecolor='gray', annot=True, fmt="d")
plt.xlabel("Student ID")
plt.ylabel("Tutor ID")
plt.title("Tutor–Student Assignment Matrix")
plt.tight_layout()
plt.show()

# Scenario 2: Balance tutor's workload while maximizing tutor's preference on tuition centre

In [ ]:
# optimizer = TutorAssignmentModel(
#         new_students=new_student_info,tutor_info=tutors,
#         existing_students=existing_student_info_filtered,
#         beta= 0.9,
#         scenario=2
#         )
# optimizer.main_process()

## Generate Reports for analysis

In [ ]:
path = './results/'
task_2_assignments = pd.read_csv(f'{path}scenario_2_assignment_results.csv')
task_2_preference_report = pd.read_csv(f'{path}scenario_2_preference_report.csv')
task_2_tutor_summary = pd.read_csv(f'{path}scenario_2_tutor_summary.csv')

## Visualisation

### Tutor Utilization

#### Bar chart: How many students are assigned to each tutor compared against tutor's max capacity

In [ ]:
plt.figure()
plt.bar(task_2_tutor_summary["TutorID"],task_2_tutor_summary["TotalAssigned"], label="Total Assigned", color="skyblue")
plt.plot(task_2_tutor_summary["TutorID"],task_2_tutor_summary["MaxCapacity"], 'r--', label="Max Capacity")
plt.xlabel("Tutor ID")
plt.ylabel("Students")
plt.title("Tutor Utilization vs. Max Capacity")
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

#### Bar Chart: Each tutor's assigned student stacked against remaining capacity

In [ ]:

# Calculate utility as fraction of max capacity
task_2_tutor_summary["UtilityPct"] = (task_2_tutor_summary["TotalAssigned"] / 
                                      task_2_tutor_summary["MaxCapacity"] * 100)

plt.figure(figsize=(10,6))

# Stacked bars
plt.bar(task_2_tutor_summary["TutorID"], task_2_tutor_summary["TotalAssigned"], 
        label="Assigned", color="skyblue")
plt.bar(task_2_tutor_summary["TutorID"], 
        task_2_tutor_summary["MaxCapacity"] - task_2_tutor_summary["TotalAssigned"], 
        bottom=task_2_tutor_summary["TotalAssigned"], 
        label="Remaining Capacity", color="lightgray")

# Overlay utility percentage on top of bars
for idx, row in task_2_tutor_summary.iterrows():
        plt.text(
                row["TutorID"], row["TotalAssigned"] + 0.2, 
                f'{row["UtilityPct"]:.0f}%', ha='center', va='bottom', fontsize=9, fontweight='bold')

plt.xlabel("Tutor ID")
plt.ylabel("Number of Students")
plt.title("Tutor Capacity vs Students Assigned (with Utility %)")
plt.legend()
plt.tight_layout()
plt.show()

#### Bar chart showing the Workload utilization against Average Tutor Utilisation

In [ ]:
# Calculate utilization as percentage
task_2_tutor_summary['Utilization'] = task_2_tutor_summary['TotalAssigned'] / task_2_tutor_summary['MaxCapacity'] * 100

plt.figure(figsize=(10,6))
bars = plt.bar(
    task_2_tutor_summary["TutorID"], 
    task_2_tutor_summary["Utilization"], 
    color="skyblue", edgecolor="black")

# Average utilization line
plt.axhline(y=task_2_tutor_summary["Utilization"].mean(), color='red', linestyle='--', label='Average Utilization')

# Add numbers on top of each bar as percentages
for bar in bars:
    height = bar.get_height()
    plt.text(
        bar.get_x() + bar.get_width()/2, height + 1, 
        f'{height:.0f}%', ha='center', va='bottom', fontsize=9)

plt.xlabel("Tutor ID")
plt.ylabel("Utilization (%)")
plt.title("Tutor Workload Utilization")
plt.ylim(0,110)  # slightly higher to accommodate labels
plt.legend()
plt.tight_layout()
plt.show()



#### Stacked Bar Chart: Total Student assigned to different Tutors browken down into Existing VS New Students

In [ ]:
plt.bar(task_2_tutor_summary["TutorID"], task_2_tutor_summary["Existing"], label="Existing", color="lightgray")
plt.bar(task_2_tutor_summary["TutorID"], task_2_tutor_summary["NewAssigned"], bottom=task_2_tutor_summary["Existing"], label="New Assigned", color="skyblue")
plt.xlabel("Tutor ID")
plt.ylabel("Number of Students")
plt.title("Task 2: Tutor Utilization (Existing + New)")
plt.legend()
plt.tight_layout()
plt.show()


### Free Capacity Distribution

#### Bar Chart showing discrete counts of tutors for each free capacity value

In [ ]:
# Count number of tutors per exact free capacity
summary_counts = task_2_tutor_summary["FreeCapacity"].value_counts().sort_index()

# Vertical bar plot
plt.figure(figsize=(8,5))
plt.bar(summary_counts.index, summary_counts.values, color="lightgreen", edgecolor="black")
plt.xlabel("Free Capacity")
plt.ylabel("Number of Tutors")
plt.title("Tutor Free Capacity Distribution (Discrete)")

plt.xticks(summary_counts.index)  # ensure all integer free capacities shown
plt.tight_layout()
plt.show()


### Preference Satisfaction by Centre

#### Bar Chart: Proportion of student–tutor assignments where the tutor’s preferred centre includes the student’s tuition centre

In [ ]:
task_2_preference_report

In [ ]:
pref_rate = task_2_preference_report["MatchPref"].mean() * 100
print(f"Overall Tutor Preference Satisfaction: {pref_rate:.1f}%")

pref_by_centre = task_2_preference_report.groupby("StudentCentre")["MatchPref"].mean().mul(100)
pref_by_centre.plot(kind='bar', color='orange')
plt.ylabel("% of Matches Meeting Tutor Preference")
plt.title("Preference Satisfaction by Tuition Centre")
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()


### Assignment Matrix

#### Visualises assignment of new students to different tutors (sanity check that 1 student - 1 tutor rule met)

In [ ]:
pivot = task_2_assignments.pivot_table(index="TutorID", columns="StudentID", aggfunc=len, fill_value=0)

plt.figure(figsize=(12,8))
sns.heatmap(pivot, cmap="Blues", cbar=True, linewidths=0.5, linecolor='gray', annot=True, fmt="d")
plt.xlabel("Student ID")
plt.ylabel("Tutor ID")
plt.title("Tutor–Student Assignment Matrix")
plt.tight_layout()
plt.show()

### Comparing Efficiency (Total tutors used) VS Avg Free Capacity (Workload Balance)

#### Bar chart comparing Scenario 1 and Scenario 2 in terms of tutors used and average free capacity (Attached the actual values below)

In [ ]:
try:
    task_2_tutor_summary = pd.read_csv("./results/scenario_2_tutor_summary.csv")
    comp = pd.DataFrame({
        "Scenario": ["1 - Efficiency", "2 - Fairness"],
        "Tutors Used": [
            (task_1_tutor_summary["TotalAssigned"] > 0).sum(),
            (task_2_tutor_summary["TotalAssigned"] > 0).sum()
        ],
        "Stadard Deviation Free Capacity": [
            task_1_tutor_summary["FreeCapacity"].std(),
            task_2_tutor_summary["FreeCapacity"].std()
        ]
    })
    comp.set_index("Scenario").plot(kind="bar", color=["skyblue","lightgreen"])
    plt.title("Scenario Comparison: Tutors Used vs Standard Deviation of Free Capacity")
    plt.ylabel("Count / Capacity")
    plt.xticks(rotation=0)
    plt.tight_layout()
    plt.show()
except FileNotFoundError:
    print("Scenario 2 data not found. Skipping scenario comparison.")

In [ ]:
comp